In [1]:
import json
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%load_ext autoreload

%autoreload 2


In [2]:
import end_to_end_profiles as e2e
import profiler
import single_model_profiles as sm_profs

In [3]:
pipeline_three_df = profiler.load_pipeline_three_systemx()

/Users/crankshaw/miniconda2/envs/plots/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/crankshaw/miniconda2/envs/plots/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Skipping max_thru_hand_tuned


In [ ]:
pipeline_three_df

In [ ]:
pipeline_one_df = profiler.load_pipeline_one_systemx()

In [ ]:
def plot_pipeline_exp(pipeline):
    plt.clf()
    fig, (ax_thru, ax_lat) = plt.subplots(ncols=2, nrows=1, sharey=True)
    fig.set_size_inches(12,5)
    colors = sns.color_palette("deep", n_colors=5)
    i = 0
    width = 0.1
    delta = 0.03
    for title, group in pipeline.groupby("name"):
        print(title)
        if "max" not in title:
            continue
        if "alexnet" in title:
            group = group.iloc[idxs_to_keep]
        box = ax_lat.boxplot(x=group["latency"], positions=group["cost"] + i*(width + delta), widths=width, patch_artist=True,
                            vert=False)
        for patch in box["boxes"]:
            patch.set_facecolor(colors[i])
        lat = group["estimated_latency"].iloc[0]
        cost_xs = np.arange(0.0, max(pipeline["cost"]) + 0.8, 0.1)
        lat_ys = lat * np.ones(len(cost_xs))
        ax_lat.plot(lat_ys, cost_xs, '--', linewidth=2, color=colors[i])
#         ax_thru.scatter(x=group["cost"], y=group["mean_throughput"], c=colors[i], marker="o", label=title)
#         ax_thru.scatter(x=group["cost"], y=group["estimated_thru"], c=colors[i], marker="^", label="")
        g_sorted = group.sort_values("cost")
        ax_thru.plot(g_sorted["mean_throughput"], g_sorted["cost"], color=colors[i], linestyle="solid", marker="o", label=title)
        ax_thru.plot(g_sorted["estimated_thru"], g_sorted["cost"], color=colors[i], linestyle="dashed", label="")

        i += 1
    ax_lat.set_xlabel("Latency (s)")
    ax_lat.set_ylim((1,max(pipeline["cost"]) + 0.8))
#     ax_thru.set_xlim((0,max(pipeline["cost"]) + 0.8))

    ax_lat.set_ylabel("Cost ($/hr)")
    ax_thru.set_xlabel("Throughput (queries/s)")
    ax_thru.set_ylabel("Cost ($/hr)")
    ax_thru.legend(loc=2)
#     ax_lat.legend(loc=2)


    plt.show()

In [ ]:
plot_pipeline_exp(pipeline_three_df)

In [ ]:
pipeline_one_df

In [ ]:
pipeline_three_df

In [ ]:
single_model_profs = sm_profs.load_single_model_profiles()

In [ ]:

single_model_profs["alexnet"].sort_values("mean_throughput_qps")

In [ ]:
single_model_profs["res50"].sort_values("mean_throughput_qps")

In [ ]:
single_model_profs["res152"].sort_values("mean_throughput_qps")

In [ ]:
probs = {'alexnet': 1.0, 'res50': 0.8077178975382568, 'res152': 0.4318030605455755}

In [ ]:
.12+.13+.12

In [ ]:
best_thruput = 0
bt_lat = 1000
bt_idxs = {"alex": -1, "res50": -1, "res152": -1}
bt_batches = {"alex": -1, "res50": -1, "res152": -1}

slo = 0.5

for ai, arow in single_model_profs["alexnet"].iterrows():
    for r50i, r50row in single_model_profs["res50"].iterrows():
        for r152i, r152row in single_model_profs["res152"].iterrows():
            total_latency = arow["p99_latency_ms"] + r50row["p99_latency_ms"] + r152row["p99_latency_ms"]
            thru = min(arow["mean_throughput_qps"], r50row["mean_throughput_qps"] / probs["res50"], r152row["mean_throughput_qps"] / probs["res152"])
            if thru > best_thruput and total_latency < slo:
                best_thruput = thru
                bt_lat = total_latency
                bt_idxs = {"alex": ai, "res50": r50i, "res152": r152i}
                bt_batches = {"alex": arow["mean_batch_size"], "res50": r50row["mean_batch_size"], "res152": r152row["mean_batch_size"]}



In [ ]:
best_thruput

In [ ]:
bt_batches

In [ ]:
bt_lat

In [ ]:
single_model_profs["res50"].sort_values("mean_throughput_qps")

In [ ]:
single_model_profs["tf-log-reg"].sort_values("mean_throughput_qps")

In [ ]:
pipeline_three_df[pipeline_three_df["name"] == "alexnet_cpu_max_thru"].sort_values("cost").iloc[idxs_to_keep]

In [ ]:
idxs_to_keep = [7, 10, 13, 0, 1]

In [15]:
import end_to_end_profiles as e2e

pipeline_one_single_proc = e2e.load_pipeline_one_single_proc()


In [17]:
pipeline_one_single_proc

,cost,mean_latency,mean_throughput,name,num_replicas,p99_latency
0,1.59,1.234579,51.836714,max_thru,1,1.310346
1,3.18,1.245383,102.786399,max_thru,2,1.354510
2,4.77,1.237378,155.221108,max_thru,3,1.345724
3,6.36,1.249625,204.898440,max_thru,4,1.367540
0,1.59,0.090452,11.048699,min_lat,1,0.100457
1,3.18,0.096901,20.635533,min_lat,2,0.107596
2,4.77,0.096239,31.158461,min_lat,3,0.108436
3,6.36,0.099542,40.162823,min_lat,4,0.113352
